# Dataset, Dataloader, BatchNorm, Dropout, Оптимизация

Сегодня у нас довольно объемная программа, она призвана закончить большую часть про базовую функциональность Pytorch, дальше мы двинемся к сверточным сетям и будем решать более прикладные задачи

Для удобства в PyTorch предоставляется ряд утилит для загрузки датасетов, их предварительной обработки и взаимодействия с ними. Эти вспомогательные классы находятся в модуле torch.utils.data module. Здесь следует обратить внимание на:

- Dataset,
- DataLoader, отвечающий за загрузку датасета.

Для создания новых датасетов наследуется класс torch.utils.data.Dataset и переопределяется метод __len__, так, чтобы он возвращал количество образцов в датасете, а также метод __getitem__ для доступа к единичному значению по конкретному индексу. Например, так выглядит простой датасет, в котором инкапсулирован диапазон целых чисел:

In [1]:
import math
import torch

class RangeDataset(torch.utils.data.Dataset):
    def __init__(self, start, end, step=1):
        self.start = start
        self.end = end
        self.step = step

    def __len__(self):
        return math.ceil((self.end - self.start) / self.step)

    def __getitem__(self, index):
        value = self.start + index * self.step
        return value

Объект DataLoader принимает датасет и ряд опций, конфигурирующих процедуру извлечения образца. Например, можно параллельно загружать образцы, задействовав множество процессов. Для этого конструктор DataLoader принимает аргумент num_workers. Обратите внимание: DataLoader всегда возвращает батчи, размер которых задается в параметре batch_size

In [2]:
dataset = RangeDataset(0, 10)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True,  drop_last=True)

for i, batch in enumerate(data_loader):
    print(i, batch)

0 tensor([4, 0, 8, 5])
1 tensor([9, 1, 7, 3])


При этом нам часто нужно делать аугментацию данных или как-то их предобрабатывать. Это можно делать внутри датасета, но частично это можно переложить и внутрь DataLoader (особенно в части обработки изображений)

In [3]:
import numpy as np

from PIL import Image
from torchvision import transforms, datasets

In [4]:
train_dataset = datasets.CIFAR10(root='data/', train=True, download=True)
image, label = train_dataset[0]

Files already downloaded and verified


In [5]:
class MyOwnCifar(torch.utils.data.Dataset):
   
    def __init__(self, init_dataset, transform=None):
        self._base_dataset = init_dataset
        self.transform = transform

    def __len__(self):
        return len(self._base_dataset)

    def __getitem__(self, idx):
        img = self._base_dataset[idx][0]
        if self.transform is not None:
            img = self.transform(img)
        img = img.flatten()
        return img, self._base_dataset[idx][1]
    
trans_actions = transforms.Compose([
                                    transforms.RandomCrop(32, padding=4), 
                                    transforms.ToTensor()])

dataset = MyOwnCifar(train_dataset, trans_actions)
train_loader = torch.utils.data.DataLoader(dataset,
                          batch_size=10,
                          shuffle=True
                        )

# Dropout, BatchNorm

Зачем? -> для улучшения сходимости и ускорения обучения

## Dropout - исключение заданного числа нейронов

<img src="dropout.png">

Посмотрим на доске как это работает. В рекурентных сетях dropout может работать немного по-другому, обсудим...

## BatchNorm

это метод, используемый для ускорения и повышения стабильности искусственных нейронных сетей за счет нормализации входных данных слоев путем повторного центрирования и масштабирования

<img src="batchnorm.png">

In [6]:
import torch.nn.functional as F
import torch.nn as nn

In [7]:
class Perceptron(nn.Module):
    def __init__(self, input_dim, output_dim, activation="relu"):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.activation = activation
        
    def forward(self, x):
        x = self.fc(x)
        if self.activation=="relu":
            return F.relu(x)
        if self.activation=="sigmoid":
            return F.sigmoid(x)
        raise RuntimeError
        

class FeedForward(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(FeedForward, self).__init__()
        self.bn1 = nn.BatchNorm1d(input_dim)
        self.fc1 = Perceptron(input_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.dp = nn.Dropout(0.25)
        self.fc2 = Perceptron(hidden_dim, 10, "relu")
        
    def forward(self, x):
        x = self.bn1(x)
        x = self.fc1(x)
        x = self.bn2(x)
        x = self.dp(x)
        x = self.fc2(x)
        return x

In [8]:
net = FeedForward(3*32*32, 200)

optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [9]:
from tqdm import tqdm


In [10]:
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]
        break

In [12]:
net(inputs)

tensor([[0.0000, 0.0000, 0.4306, 0.0000, 0.1269, 0.0000, 0.1023, 0.0000, 0.0316,
         0.8255],
        [0.0000, 0.2348, 0.0000, 1.4373, 0.0000, 0.3017, 0.0000, 0.0174, 0.0000,
         0.0000]], grad_fn=<ReluBackward0>)

In [ ]:
for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 300))
            running_loss = 0.0

print('Training is finished!')

# Оптимизация нейронных сетей

Подходов к оптимизации нейронных сетей существует огромное количество. Мы посмотрим на самые популярные из них. Главное понимать основное направление мысли в улучшениях:

<img src="Optims.gif">

## SGD

<img src="SGD.png">

## SGD + Momentum

<img src="SGD_momentum.png">

## Adagrad

<img src="Adagrad1.png">
<img src="Adagrad2.png">

## RMSProp

<img src="RMSProp.png">

## Adam

<img src="Adam1.png">
<img src="Adam2.png">
<img src="Adam3.png">

In [ ]:
# Запустим сеть с разными оптимизаторами

In [ ]:
optimizer = torch.optim.Adam(net.parameters())
criterion = nn.CrossEntropyLoss()

In [ ]:
Динамические vs статические

In [ ]:
for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 300))
            running_loss = 0.0

print('Training is finished!')